In [3]:
import os

In [4]:
%pwd

'd:\\Code\\Git\\myprojects\\Equation-Solver-End-to-End\\Handwritten_Equation_Solver\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Code\\Git\\myprojects\\Equation-Solver-End-to-End\\Handwritten_Equation_Solver'

In [13]:
import dagshub
dagshub.init(repo_owner='anandsr724', repo_name='Handwritten_Equation_Solver', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2024-11-05 16:22:50,642: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/anandsr724/Handwritten_Equation_Solver "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "anandsr724/Handwritten_Equation_Solver"

[2024-11-05 16:22:50,647: INFO: helpers: Initialized MLflow to track repo "anandsr724/Handwritten_Equation_Solver"]


Repository anandsr724/Handwritten_Equation_Solver initialized!

[2024-11-05 16:22:50,649: INFO: helpers: Repository anandsr724/Handwritten_Equation_Solver initialized!]


2024/11/05 16:22:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run caring-deer-855 at: https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow/#/experiments/0/runs/f94401e4a0604bf2848fd543702fe9d2.
2024/11/05 16:22:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow/#/experiments/0.


In [14]:
import tensorflow as tf

[2024-11-05 16:23:03,931: WARNING: module_wrapper: From d:\Code\Git\myprojects\Equation-Solver-End-to-End\Handwritten_Equation_Solver\.eq_solver_venv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [15]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

[2024-11-05 16:23:04,991: WARNING: module_wrapper: From d:\Code\Git\myprojects\Equation-Solver-End-to-End\Handwritten_Equation_Solver\.eq_solver_venv\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]


In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [17]:
from equation_solver.constants import *
from equation_solver.utils.common import read_yaml, create_directories, save_json

In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/symbol_data",
            mlflow_uri="https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [19]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [20]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        input_shape = self.model.input_shape
        color_mode = "grayscale" if input_shape[-1] == 1 else "rgb"
        # print(f"Using color mode: {color_mode}")

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            color_mode=color_mode,
            class_mode='sparse'  # For sparse categorical crossentropy
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Validation generator - no shuffling
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()

        # self.score = model.evaluate(self.valid_generator)
        self.score = self.model.evaluate(self.valid_generator)
        
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.tensorflow.log_model(self.model, "model", registered_model_name="Model768")
            else:
                mlflow.tensorflow.log_model(self.model, "model")
                  # Get TensorFlow version for logging
            # tf_version = tf.__version__
            
            # # Log as a TensorFlow model instead of Keras
            # if tracking_url_type_store != "file":
            #     mlflow.tensorflow.log_model(
            #         model=self.model,
            #         artifact_path="model",
            #         registered_model_name="Model768",
            #         metadata={
            #             "tensorflow_version": tf_version,
            #             "framework": "TensorFlow"
            #         }
            #     )
            # else:
            #     mlflow.tensorflow.log_model(
            #         model=self.model,
            #         artifact_path="model",
            #         metadata={
            #             "tensorflow_version": tf_version,
            #             "framework": "TensorFlow"
            #         }
            #     )


In [21]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-11-05 16:23:21,672: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-05 16:23:21,674: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-05 16:23:21,676: INFO: common: created directory at: artifacts]
Found 42589 images belonging to 25 classes.
[2024-11-05 16:23:23,739: WARNING: module_wrapper: From d:\Code\Git\myprojects\Equation-Solver-End-to-End\Handwritten_Equation_Solver\.eq_solver_venv\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
1331/1331 [==============================] - 18s 13ms/step - loss: 2.8404 - accuracy: 0.1596
[2024-11-05 16:23:41,604: INFO: common: json file saved at: scores.json]


2024/11/05 16:23:42 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-11-05 16:23:43,805: INFO: builder_impl: Assets written to: C:\Users\anand\AppData\Local\Temp\tmp3fzsrgs9\model\data\model\assets]


2024/11/05 16:23:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Model768' already exists. Creating a new version of this model...
2024/11/05 16:24:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Model768, version 4
Created version '4' of model 'Model768'.
2024/11/05 16:24:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-whale-476 at: https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow/#/experiments/0/runs/88dee599204c4c01b645bfdc5f0ceb79.
2024/11/05 16:24:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/anandsr724/Handwritten_Equation_Solver.mlflow/#/experiments/0.
